# CAS Data Engineering FHNW
## Data Wrangling Module - Author F.Burnelli

- Find Duplicates Rows Exact Match
- Candidates duplicates
  - After harmonisation
  - similarity matches   

### Exercise 4
### Note:use the Kernek: "Python (wrangling)"

## Extract Data

In [1]:
import pandas as pd
import os
import sys   
# not exact match search
from thefuzz import fuzz,process
import difflib
# regexp
import re

In [2]:
## Extract Data
file_path = os.path.join("..","data", "dirty-loan-data.csv")


df = pd.read_csv(file_path, low_memory=False)

In [3]:
df = df[['emp_title']]
df = df.dropna(subset=['emp_title'])

In [4]:
df.head(10)

,emp_title
0,cognizant technology solutions
1,Anadarko Petroleum Corporation
2,Best Buy
4,"Cardinal Health Stafford, Texas."
5,St. Rose San Martin
6,cardinal logistics
7,Lifecell
8,"Herdman, Summers, and George, LLP"
9,CNMI Government
10,TOGETHER WE GROW


### How many distinct values

In [5]:
df['emp_title'].nunique()

10654

### Find Duplicates  (and remove)
HINT duplicated().sum() ; .drop_duplicates()

### Harmonise (and remove duplicated again)
HINT  str.lower ; strip ; regexp or replace non alphanumerics

In [7]:
df['emp_title'].nunique()

9384

In [9]:
df.head(3)

,emp_title
0,cognizant technology solutions
1,anadarko petroleum corporation
2,best buy


# Similarity Search


#### How does thefuzz works?

In [58]:
process.extract("cognizant technology solutions", df['emp_title'], limit = 10)

[('cognizant technology solutions', 100, 0),
 ('at&t', 68, 32),
 ('life technologies', 61, 57),
 ('u store it', 51, 44),
 ('southwire', 50, 70),
 ('southwest airlines', 47, 43),
 ('orly international', 47, 69),
 ('cardinal logistics', 46, 5),
 ('orgill inc.', 46, 72),
 ('zila inc.', 45, 17)]

### thefuzz process (NOTE: just first 100 rows ...)
Find a useful treshold

In [16]:
# Match
TRESHOLD=80
def matching(reference_string,values,limit=5,treshold=80):
    #the fuzzy match
    similarities = process.extract(reference_string, values, limit = limit)
    matches= [item for item in similarities if treshold < item[1] < 100]
    res = f"{reference_string}:"
    for match in matches:
        res = res + f"{match[0]} (SCORE:{match[1]});"
    print(res)
    print("\n")

values = df['emp_title'].unique()
values.sort()
for value in values[0:100]:
    matching(value,values,treshold=TRESHOLD)  


1 lpn 2 driver:drive (SCORE:90);driver (SCORE:90);lpn (SCORE:90);account 1 speciallist (SCORE:86);


10:et level 10 (SCORE:90);gs10 (SCORE:90);peak 10 (SCORE:90);


1st assistant manager:assistant manager (SCORE:95);assistant (SCORE:90);manage (SCORE:90);manager (SCORE:90);


1st grade teachergrade level lead:teach (SCORE:90);teacher (SCORE:90);1st assistant manager (SCORE:86);1st sergeant (SCORE:86);


1st sergeant:sergeant (SCORE:90);1st assistant manager (SCORE:86);1st grade teachergrade level lead (SCORE:86);correction s training sergeant (SCORE:86);


21st century security:security (SCORE:90);armed security officer us federal bldgs (SCORE:86);building automated security technician (SCORE:86);clients security fund coordinator (SCORE:86);


24 hour fitness:it (SCORE:90);la fitness (SCORE:86);us deparment of labor wage and hour di (SCORE:86);


245 e realty corp:clp corp (SCORE:86);corp manager  finanial systems (SCORE:86);e line (SCORE:86);harris corp (SCORE:86);


26726 crown vahea

### SequenceMatcher

In [17]:
matcher = difflib.SequenceMatcher(None,"full_name_reordered", "full_name")
similarity_ratio = matcher.ratio() * 100
similarity_ratio

64.28571428571429

In [24]:
def matching(reference_string,values):
    for value in values:
        matcher = difflib.SequenceMatcher(None,reference_string,value)
        score = matcher.ratio() * 100
        if 100 > score > 80  :
                print(reference_string,value,score)
        
values = df['emp_title'].unique()
values.sort()
for value in ["teacher"]:# values[0:100]:
    matching(value,values)

teacher teach 83.33333333333334
teacher teachers 93.33333333333333


### Business 
Sometimes business rules can helps ,
- is RN actually REGISTERED NURSE?
- should we just generalise "manager" role?

In [23]:
df['emp_title'].value_counts().head(10)

emp_title
teacher             341
manager             297
registered nurse    171
owner               152
rn                  142
supervisor          131
driver              126
office manager      113
sales               107
general manager      94
Name: count, dtype: int64

407